In [2]:
import os
os.chdir("/home")


In [4]:
from random import sample
from tool import norm_edit_dis
from pathlib import Path
infer_path = "/home/works/rec_PPOCR_sangmu_and_outsourcing_final_v2/inference_result"
label_path = "/home/labelsets/sangmu_str_only_test/test_label.txt"
n = 4

## infer 결과와 레이블 정보 결합
with open(label_path) as f:
    lines = [line.rstrip("\n").split("\t") for line in f.readlines()]
    label_dict = {}
    for path, label in lines:
        label_dict["/home/datasets/"+path] = label

with open(infer_path) as f:
    lines = [line.rstrip("\t").split("\t") for line in f.readlines()]
    lines = [[path, pred, conf, label_dict[path], norm_edit_dis(pred, label_dict[path])] for path, pred, conf in lines]

# norm_edit에 따라 구간 별로 구분 (0.1, 0.2, ... 1)
samples = [[] for i in range(11)]
for path, pred, conf, label, dis in lines:
    idx = int(float(dis)*10)
    samples[idx].append([path, pred, conf, label, dis])

# 세트 구분
samples = sum(samples, [])
batches = [[] for i in range(n)]
for i, sample in enumerate(samples):
    idx = i%n
    batches[idx].append(sample)
    
# 결과 요약 출력
print("**세트 별 개수와 norm_dis 평균 출력**")
for i, batch in enumerate(batches):
    print(f"set {i} => sample num = {len(batch)}, norm_dis = {sum([float(x[4]) for x in batch])/len(batch)} ", )


for i, batch in enumerate(batches):
    with open(f"batch_{i}.txt", "w") as f:
        for path, pred, conf, label, dis in batch:
            path = "/".join(path.split("/")[3:])  
            f.write(f"{path}\t{label}\n")
    

**세트 별 개수와 norm_dis 평균 출력**
set 0 => sample num = 1385, norm_dis = 0.22915457372309336 
set 1 => sample num = 1385, norm_dis = 0.22896338093037533 
set 2 => sample num = 1385, norm_dis = 0.22856639781548307 
set 3 => sample num = 1384, norm_dis = 0.22918631388064212 
